In [1]:
%matplotlib widget
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.spines['top'].set_visible(False) 

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [2]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

In [3]:
latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir

df = pd.read_csv(latestdir+'/Regionalt_DB/24_reinfektioner_daglig_region.csv',encoding='latin1',delimiter = ';')
df['Prøvedato'] = pd.to_datetime(df['Prøvedato'])
# groupdf = df.groupby(['Prøvedato').sum()
df_reinf = df[df['Type af tilfælde (reinfektion eller bekræftet tilfælde)'] == '1.Reinfektion'].groupby('Prøvedato').sum()
df_inf = df[df['Type af tilfælde (reinfektion eller bekræftet tilfælde)'] != '1.Reinfektion'].groupby('Prøvedato').sum()
# df_reinf
# groupdf = df.groupby(['Prøvedato', 'Type af tilfælde (reinfektion eller bekræftet tilfælde)']).sum()

# kortdf  = pd.read_csv(latestdir+'/Regionalt_DB/13_regionale_kort.csv',encoding='latin1',delimiter = ';')
# kortdf

In [4]:
fig,ax1 = plt.subplots()

# ax1.plot(df_inf.index,df_inf.infected,'.:')
# ax1.plot(df_reinf.index,df_reinf.infected,'.:')

meanWidth=7
ax1.plot(df_inf.index,df_inf.infected,'k.:',markersize=4,linewidth=0.5,label='Alle infektioner')
ax1.plot(rnTime(df_inf.index,meanWidth),rnMean(df_inf.infected,meanWidth),'k',label=f'Alle infektioner, {meanWidth} dages gennemsnit')
ax1.plot(df_reinf.index,df_reinf.infected,'b.:',markersize=4,linewidth=0.5,label='Reinfektion')
ax1.plot(rnTime(df_reinf.index,meanWidth),rnMean(df_reinf.infected,meanWidth),'b',label=f'Reinfektion, {meanWidth} dages gennemsnit')
ax1.plot(df_reinf.index,df_inf.infected-df_reinf.infected,'m.:',markersize=4,linewidth=0.5,label='Nye infektioner')
ax1.plot(rnTime(df_reinf.index,meanWidth),rnMean(df_inf.infected-df_reinf.infected,meanWidth),'m',label=f'Nye infektioner, {meanWidth} dages gennemsnit')

firstDate = np.datetime64('2021-10-01')
ax1.set_xlim(left=firstDate)
ax1.legend(loc='upper left')
ax1.grid()
ax1.set_ylim(bottom=0)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax1.spines['top'].set_visible(False) 
ax1.spines['right'].set_visible(False)

ax1.set_ylabel('Antal infektioner')

if saveFigures:
    fig.savefig(path_figs+'Reinfections/Reinfections_Overview')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,tight_layout=True)

curDates = df_inf.index.values 
newInf = df_inf.infected.values-df_reinf.infected.values

# ax1.plot(df_inf.index,df_inf.infected,'k.:',markersize=4,linewidth=0.5,label='Alle infektioner')
# ax1.plot(rnTime(df_inf.index,meanWidth),rnMean(df_inf.infected,meanWidth),'k',label=f'Alle infektioner, {meanWidth} dages gennemsnit')
# ax1.plot(df_reinf.index,df_reinf.infected,'b.:',markersize=4,linewidth=0.5,label='Reinfektion')
# ax1.plot(rnTime(df_reinf.index,meanWidth),rnMean(df_reinf.infected,meanWidth),'b',label=f'Reinfektion, {meanWidth} dages gennemsnit')
# ax1.plot(curDates,newInf,'k.:',markersize=4,linewidth=0.5,label='Nye infektioner')
# ax1.plot(rnTime(curDates,meanWidth),rnMean(newInf,meanWidth),'m',label=f'Nye infektioner, {meanWidth} dages gennemsnit')
ax1.fill_between(rnTime(curDates,meanWidth),rnMean(newInf,meanWidth),color='k',label=f'Nye infektioner')
ax1.fill_between(rnTime(curDates,meanWidth),rnMean(newInf,meanWidth),rnMean(df_inf.infected.values,meanWidth),color='xkcd:darkgreen',label=f'Reinfektioner')

curRatio = rnMean(df_inf.infected.values-newInf,meanWidth)/rnMean(df_inf.infected.values,meanWidth)
ax2.fill_between(rnTime(curDates,meanWidth),100*np.ones(rnMean(df_inf.infected.values-newInf,meanWidth).shape),color='k')
ax2.fill_between(rnTime(curDates,meanWidth),100*curRatio,color='xkcd:darkgreen')

ax1.set_xlim(left=firstDate)
ax1.legend(loc='upper left')
ax1.set_axisbelow(True)
ax1.grid()
ax2.grid(axis='y')
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=np.max(curRatio)*1.2*100)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax1.spines['top'].set_visible(False) 
ax1.spines['right'].set_visible(False)

ax1.set_ylabel('Antal infektioner\n(7 dages gennemsnit)')
ax2.set_ylabel('Andel af infektioner [%]\n(7 dages gennemsnit)')

if saveFigures:
    fig.savefig(path_figs+'Reinfections/Reinfections_Stacked')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# # curdf = groupdf.loc[allRegions[1],:,:]
# # curDates = groupdf.index.get_level_values(0)
# # curCase = curdf['Bekræftede tilfælde']
# # curAdm = curdf['Indlæggelser']
# # curDea = curdf['Døde']
# # curDates
# groupdf